<a href="https://colab.research.google.com/github/AFirooz/bayesian-network-analysis-tools/blob/main/Monty_Hall_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -U: upgrade all
# -q: quite install

!apt-get install -yq graphviz graphviz-dev
!pip install -Uq pgmpy graphviz pygraphviz

Reading package lists...
Building dependency tree...
Reading state information...
graphviz is already the newest version (2.42.2-6ubuntu0.1).
libgraphviz-dev is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


# Building the Model

We'll have the attributes below with states denoting the door number.

1. **Contestant:** `C ∈ {1, 2, 3}`
2. **Host:** `H ∈ {1, 2, 3}`
3. **Prize:** `P ∈ {1, 2, 3}`

In [4]:
from IPython.display import Image
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD


# Defining the network structure
model = DiscreteBayesianNetwork([
    ("C", "H"),
    ("P", "H")
])

# Defining the CPDs:
cpd_c = TabularCPD("C", 3, [[0.33], [0.33], [0.33]])
cpd_p = TabularCPD("P", 3, [[0.33], [0.33], [0.33]])
cpd_h = TabularCPD("H", 3,
    [[0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
     [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
     [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0]],
    evidence=["C", "P"],
    evidence_card=[3, 3],
)

# Associating the CPDs with the network structure.
model.add_cpds(cpd_c, cpd_p, cpd_h)

# check the model structure and associated CPDs
model.check_model()

[<TabularCPD representing P(C:3) at 0x796162986cd0>,
 <TabularCPD representing P(P:3) at 0x796162986c10>,
 <TabularCPD representing P(H:3 | C:3, P:3) at 0x796162986dd0>]

In [6]:
# Some other methods
all_cpd = model.get_cpds()

for cpd in all_cpd:
    print(cpd, '\n')

+------+------+
| C(0) | 0.33 |
+------+------+
| C(1) | 0.33 |
+------+------+
| C(2) | 0.33 |
+------+------+ 

+------+------+
| P(0) | 0.33 |
+------+------+
| P(1) | 0.33 |
+------+------+
| P(2) | 0.33 |
+------+------+ 

+------+------+------+------+------+------+------+------+------+------+
| C    | C(0) | C(0) | C(0) | C(1) | C(1) | C(1) | C(2) | C(2) | C(2) |
+------+------+------+------+------+------+------+------+------+------+
| P    | P(0) | P(1) | P(2) | P(0) | P(1) | P(2) | P(0) | P(1) | P(2) |
+------+------+------+------+------+------+------+------+------+------+
| H(0) | 0.0  | 0.0  | 0.0  | 0.0  | 0.5  | 1.0  | 0.0  | 1.0  | 0.5  |
+------+------+------+------+------+------+------+------+------+------+
| H(1) | 0.5  | 0.0  | 1.0  | 0.0  | 0.0  | 0.0  | 1.0  | 0.0  | 0.5  |
+------+------+------+------+------+------+------+------+------+------+
| H(2) | 0.5  | 1.0  | 0.0  | 1.0  | 0.5  | 0.0  | 0.0  | 0.0  | 0.0  |
+------+------+------+------+------+------+------+--

# Infering the Posterior Probability


In [7]:
from pgmpy.inference import VariableElimination


infer = VariableElimination(model)
posterior_p = infer.query(["P"], evidence={"C": 0, "H": 2})

print(posterior_p)

+------+----------+
| P    |   phi(P) |
+======+==========+
| P(0) |   0.3333 |
+------+----------+
| P(1) |   0.6667 |
+------+----------+
| P(2) |   0.0000 |
+------+----------+
